### Setup

In [ ]:
import json

import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from scipy.spatial import ConvexHull
import darkdetect

if darkdetect.isDark():
    plt.style.use('dark_background')
else:
    plt.style.use('default')

### Loading data

In [ ]:
with open('../generated/pilchowice/data.json') as f:
    data = json.load(f)

places: list[tuple[str, float, float]] = []
for section in data['sections']:
    for place in section['places']:
        places.append((place['id'], place['lng'], place['lat']))

coords = np.array([(lng, lat) for _, lng, lat in places])
hull = ConvexHull(coords)

print(f'center: {coords.mean(0)[0]}')

### Printing coordinates of places that most far away

In [ ]:
bounding_indices = np.unique(hull.simplices.flat)

bounding_places = [places[i] for i in bounding_indices]
for i, place in enumerate(bounding_places):
    print(f'vertex {i}, {place[0]}, {place[1]}, {place[2]}')

### Chart

In [ ]:
bounding_coords = np.array([(place[1], place[2]) for place in bounding_places])
lngs = np.take(bounding_coords, 0, axis=1)
lats = np.take(bounding_coords, 1, axis=1)
for_plot = np.vstack([lngs, lats])
points = np.array(coords)

fig, ax = plt.subplots()
ax.scatter(points[:,0], points[:,1])
for i, simplex in enumerate(hull.simplices):
    lng = points[simplex, 0]
    lat = points[simplex, 1]
    ax.plot(lng, lat, 'r-')

    if np.array(simplex[0], simplex[1]) in bounding_indices:
        ax.annotate(f'{bounding_places[i][0]}', xy=(for_plot[0, i], for_plot[1, i]), color='orange')